In [ ]:
import datetime as dt

import numpy as np

import matplotlib as plt

from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.circuit.library import TwoLocal
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, Session, IBMRuntimeError
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.units import DistanceUnit

from scipy.optimize import minimize

import pyscf

In [ ]:
use_simulator = True

In [ ]:
service = QiskitRuntimeService()
hardware = service.backend(name='ibm_rensselaer')
simulator = AerSimulator.from_backend(backend=hardware)

if use_simulator:
  backend = simulator
else:
  backend = hardware

In [ ]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

#### The geometric structure of water
![water.png](water.png)

In [ ]:
def create_h2o_molecule_driver(bond_length=95.84, angle=np.radians(104.45)):
  oxygen = f"O 0 0 0"
  hydrogen1 = f"H {-1 * np.sin(angle) * bond_length} {-1 * np.cos(angle) * bond_length} 0"
  hydrogen2 = f"H {np.sin(angle) * bond_length} {-1 * np.cos(angle) * bond_length} 0"

  water = "; ".join([oxygen, hydrogen1, hydrogen2])

  driver = PySCFDriver(
    atom=water,
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM
  )

  return driver

In [ ]:
bond_length=95.84
angle=np.radians(104.45)

oxygen = f"O 0 0 0"
hydrogen1 = f"H {-1 * np.sin(angle) * bond_length} {-1 * np.cos(angle) * bond_length} 0"
hydrogen2 = f"H {np.sin(angle) * bond_length} {-1 * np.cos(angle) * bond_length} 0"

water = "; ".join([oxygen, hydrogen1, hydrogen2])

driver = PySCFDriver(
  atom=water,
  basis="sto3g",
  charge=0,
  spin=0,
  unit=DistanceUnit.ANGSTROM
)